In [ ]:
# This code I used to prompt Qwen/Qwen2.5-Math-7B-Instruct to generate solutions for computational questions 1, ..., 30

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!pip install transformers huggingface_hub
!pip install -q keras

In [ ]:
from huggingface_hub import login
import json
import re
from pathlib import Path
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import os

GoogleDrive= "/content/drive/MyDrive"

Prompts = os.path.join(GoogleDrive, "ANT", "Prompts Alg NT")    #to use this code you must change this directory
Save_Outputs = os.path.join(Prompts, "outputs for questions")   #to use this code you must change this directory

os.makedirs(Save_Outputs)
os.makedirs("./offload")

Hinting_Strategies = ["Few-shot Hinting", "Chain-of-Thought (CoT) Prompting", "Automatic Chain-of-Thought (Auto-CoT) Prompting", "Self-Consistency", "Logical Chain-of-Thought (LogiCoT) Prompting", "Chain-of-Symbol (CoS) Prompting", "Structured Chain-of-Thought (SCoT) Prompting", "ReAct Prompting", "Clear and Specific Instructions"]


In [ ]:
Hugging_Face_token= "..." # put your hugging face token to be able to use this

Model_ID   = "Qwen/Qwen2.5-Math-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(Model_ID)
tokenizer = AutoTokenizer.from_pretrained(Model_ID)
pipe= pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
for j in range(1, 31):
    docj= (Prompts / f"Question {j}.txt").read_text(encoding="utf-8")
    question= re.search(r"\\subsection\*\{Question\}(.*?)\\subsection\*\{Hint(?:s)?\}", docj, re.DOTALL).group(1).strip()
    Nine_Hints=  re.search(r"\\subsection\*\{Hint(?:s)?\}(.*?)(\\subsection\*\{Answer/Algorithm\}|$)", docj, re.DOTALL).group(1)
    hints= re.findall(r"\\subsection\*\{([^}]+)\}(.*?)(?=\\subsection\*\{|$)", Nine_Hints, re.DOTALL)
    answers = {}
    for i, (method, hint_i) in enumerate(hints, start=1):
        prompt = (
            f"### Computational Question {j}\n\n"
            f"**Question:** {question}\n\n"
            f"**Hint ({method}):** {hint_i.strip()}\n\n"
            f"**Generated Answer:**"
        )
        output = pipe(prompt, max_new_tokens=2048, do_sample=False)[0]["generated_text"]
        answer= output[len(prompt):].strip()
        answers[f"Hint_{i}"] = {
            "method": method,
            "prompt": prompt,
            "generated_answer": answer
        }
    outputs = os.path.join(Save_Outputs, f"Question_{j}_outputs.json")
    with open(outputs, "w", encoding="utf-8") as f:json.dump(answers, f, indent=2)
print("done")

